In [1]:
print("hii")

hii


In [ ]:
import streamlit as st
import os
from git import Repo, GitCommandError
from pathlib import Path
import logging
from typing import List, Optional
import requests  # For API calls

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DocumentationGenerator:
    """Handles the generation of documentation for code repositories."""
    
    def __init__(self, api_url: str, api_key: str):
        """Initialize the documentation generator with API settings."""
        self.api_url = api_url
        self.api_key = api_key
        self.supported_extensions = {
            ".py": "Python",
            ".js": "JavaScript",
            ".java": "Java",
            ".cpp": "C++",
            ".html": "HTML"
        }
    
    def clone_repository(self, repo_url: str, local_path: str = "cloned_repo") -> str:
        """
        Clone a GitHub repository to a local directory with error handling.
        
        Args:
            repo_url: URL of the GitHub repository
            local_path: Local directory to clone into
            
        Returns:
            Status message indicating success or failure
        """
        try:
            if os.path.exists(local_path):
                if os.path.exists(os.path.join(local_path, ".git")):
                    logger.info("Repository already exists locally")
                    return "Repository already exists. Using existing clone."
                else:
                    os.rename(local_path, f"{local_path}_backup_{int(time.time())}")
            
            Repo.clone_from(repo_url, local_path)
            logger.info(f"Successfully cloned repository to {local_path}")
            return f"Repository cloned successfully to {local_path}"
            
        except GitCommandError as e:
            error_msg = f"Failed to clone repository: {str(e)}"
            logger.error(error_msg)
            return error_msg
    
    def extract_code_files(self, repo_path: str) -> List[Path]:
        """
        Extract all supported code files from the repository.
        
        Args:
            repo_path: Path to the repository
            
        Returns:
            List of paths to code files
        """
        code_files = []
        try:
            for path in Path(repo_path).rglob("*"):
                if path.suffix in self.supported_extensions:
                    if ".git" not in str(path) and "node_modules" not in str(path):
                        code_files.append(path)
            
            logger.info(f"Found {len(code_files)} code files")
            return code_files
            
        except Exception as e:
            logger.error(f"Error extracting code files: {str(e)}")
            return []
    
    def analyze_code(self, file_path: Path) -> Optional[str]:
        """
        Generate documentation for a given code file using the Grok or Gemini API.
        
        Args:
            file_path: Path to the code file
            
        Returns:
            Generated documentation or None if analysis fails
        """
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                code = file.read()
            
            language = self.supported_extensions.get(file_path.suffix, "Unknown")
            payload = {
                "prompt": f"""
                Analyze this {language} code file and provide comprehensive documentation including:

                # {file_path.name}

                ## Overview
                [Provide a brief description of the file's purpose]

                ## Key Components
                [List and describe main functions, classes, or modules]

                ## Dependencies
                [List external dependencies and important imports]

                ## Design Patterns
                [Identify any design patterns or architectural approaches used]

                ## Usage Examples
                [Provide example usage where appropriate]

                ## Code Quality Notes
                [Note any potential improvements or concerns]

                Code to analyze:
                ```{language.lower()}
                {code}
                ```
                """
            }

            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }

            response = requests.post(self.api_url, json=payload, headers=headers)
            response.raise_for_status()  # Raise error for non-200 responses

            documentation = response.json().get("documentation", response.text)
            logger.info(f"Generated documentation for {file_path.name}")
            return documentation
            
        except Exception as e:
            logger.error(f"Error analyzing {file_path}: {str(e)}")
            return None
    
    def save_documentation(self, file_path: Path, documentation: str,
                         output_dir: str = "generated_docs") -> Optional[Path]:
        """
        Save generated documentation to a Markdown file.
        
        Args:
            file_path: Original code file path
            documentation: Generated documentation content
            output_dir: Directory to save documentation
            
        Returns:
            Path to saved documentation file or None if save fails
        """
        try:
            output_path = Path(output_dir)
            output_path.mkdir(parents=True, exist_ok=True)
            
            doc_file = output_path / f"{file_path.stem}_documentation.md"
            doc_file.write_text(documentation)
            
            logger.info(f"Saved documentation to {doc_file}")
            return doc_file
            
        except Exception as e:
            logger.error(f"Error saving documentation: {str(e)}")
            return None
    
    def push_to_github(self, repo_path: str, branch_name: str = "auto-docs") -> str:
        """
        Push generated documentation to GitHub.
        
        Args:
            repo_path: Path to the repository
            branch_name: Name of the branch to push to
            
        Returns:
            Status message indicating success or failure
        """
        try:
            repo = Repo(repo_path)
            current = repo.active_branch.name
            
            # Create new branch if it doesn't exist
            if branch_name not in repo.heads:
                repo.create_head(branch_name)
            
            repo.heads[branch_name].checkout()
            
            # Stage documentation files
            repo.index.add(["generated_docs/"])
            repo.index.commit("Add AI-generated documentation")
            
            # Push to remote
            origin = repo.remote(name="origin")
            origin.push(branch_name)
            
            # Return to original branch
            repo.heads[current].checkout()
            
            logger.info(f"Successfully pushed documentation to branch '{branch_name}'")
            return f"Documentation pushed to GitHub branch '{branch_name}'"
            
        except Exception as e:
            error_msg = f"Error pushing to GitHub: {str(e)}"
            logger.error(error_msg)
            return error_msg

def main():
    """Main Streamlit application."""
    st.set_page_config(
        page_title="AI Code Documentation Generator",
        page_icon="📚",
        layout="wide"
    )
    
    st.title("📚 AI Code Documentation Generator")
    st.markdown("""
    This application analyzes GitHub repositories and generates comprehensive documentation
    using AI. Simply provide a repository URL to get started.
    """)
    
    # Initialize the documentation generator
    api_url = st.text_input("Enter Grok or Gemini API URL")
    api_key = st.text_input("Enter API Key", type="password")
    doc_gen = DocumentationGenerator(api_url=api_url, api_key=api_key)
    
    # Repository URL input
    repo_url = st.text_input(
        "GitHub Repository URL",
        placeholder="https://github.com/username/repository"
    )
    
    if repo_url and api_url and api_key:
        col1, col2, col3 = st.columns(3)
        
        with col1:
            if st.button("1. Clone Repository"):
                with st.spinner("Cloning repository..."):
                    result = doc_gen.clone_repository(repo_url)
                    st.info(result)
        
        with col2:
            if st.button("2. Generate Documentation"):
                with st.spinner("Analyzing code and generating documentation..."):
                    code_files = doc_gen.extract_code_files("cloned_repo")
                    
                    if not code_files:
                        st.error("No supported code files found in repository.")
                        return
                    
                    progress_bar = st.progress(0)
                    for i, file_path in enumerate(code_files):
                        documentation = doc_gen.analyze_code(file_path)
                        if documentation:
                            doc_gen.save_documentation(file_path, documentation)
                        progress_bar.progress((i + 1) / len(code_files))
                    
                    st.success(f"Documentation generated for {len(code_files)} files")
        
        with col3:
            if st.button("3. Push to GitHub"):
                with st.spinner("Pushing documentation to GitHub..."):
                    result = doc_gen.push_to_github("cloned_repo")
                    st.info(result)
    
    # Show documentation preview
    if os.path.exists("generated_docs"):
        st.markdown("### Generated Documentation Preview")
        for doc_file in Path("generated_docs").glob("*.md"):
            with st.expander(f"📄 {doc_file.stem}"):
                st.markdown(doc_file.read_text())

if __name__ == "__main__":
    main()


In [7]:
import requests

# Updated GitHub repository URL
repo_url = "https://api.github.com/repos/charanvelugu/potato-leaf-disease-detection_using-GENAI/contents"

# Function to fetch files from the GitHub repository
def fetch_github_repo_files(repo_url):
    response = requests.get(repo_url)
    if response.status_code == 200:
        return response.json()  # Returns a list of files and directories in the repo
    else:
        print(f"Failed to retrieve repository data, Status Code: {response.status_code}")
        return []

# Function to download and print Python code and text files
def download_files(repo_data):
    for file in repo_data:
        if file['name'].endswith('.py') or file['name'].endswith('.md') or file['name'].endswith('.txt'):
            file_url = file['download_url']
            file_name = file['name']
            
            try:
                file_response = requests.get(file_url)
                if file_response.status_code == 200:
                    print(f"Extracted content from {file_name}:\n")
                    print(file_response.text)
                    print("\n\n")
                    # Optionally, save the file locally
                    with open(file_name, 'w') as f:
                        f.write(file_response.text)
                else:
                    print(f"Failed to fetch {file_name}, Status Code: {file_response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error downloading {file_name}: {e}")

# Get repository files
repo_data = fetch_github_repo_files(repo_url)

# Download and print Python code and text files
download_files(repo_data)


Extracted content from potato.py:

### Health Management APP
from dotenv import load_dotenv

load_dotenv() ## load all the environment variables

import streamlit as st
import os
import google.generativeai as genai
from PIL import Image

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

## Function to load Google Gemini Pro Vision API And get response

def get_gemini_repsonse(input,image,prompt):
    model=genai.GenerativeModel('gemini-pro-vision')
    response=model.generate_content([input,image[0],prompt])
    return response.text

def input_image_setup(uploaded_file):
    # Check if a file has been uploaded
    if uploaded_file is not None:
        # Read the file into bytes
        bytes_data = uploaded_file.getvalue()

        image_parts = [
            {
                "mime_type": uploaded_file.type,  # Get the mime type of the uploaded file
                "data": bytes_data
            }
        ]
        return image_parts
    else:
        raise FileNotFoundError("No f